In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import tree

In [2]:
df = pd.read_csv("ivv_normal.csv")
df['ret'] = df['close'].pct_change()

#set intial values of lables to 0, will change into -1 or 1 later
df['label1'] = 0

In [3]:
#lableing: if return > 0, label as 1, else, label as -1
for i in range(0,len(df)):
    if df['ret'][i] > 0:
        df['label1'][i] = 1
    elif df['ret'][i] < 0:
        df['label1'][i] = -1


/home/shiyun/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/shiyun/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
df.head()

,Unnamed: 0,date,low,high,close,volume,open,close-open,close-open_%of_close,high-low,high-low_%of_close,macd,rsi,50d_avg,100d_avg,200d_avg,so,ret,label1
0,200,20010307,-0.439921,-0.455396,126.55,-0.669027,-0.438876,-0.124481,-0.119906,-0.293656,-0.254819,-1.713257,-0.062340,-0.328303,-0.265805,-0.107318,-0.765715,NaN,0
1,201,20010308,-0.429719,-0.447085,126.92,-1.015534,-0.439084,0.174580,0.180720,-0.327246,-0.289116,-1.572642,0.318968,-0.330083,-0.267201,-0.108762,-0.303249,0.002924,1
2,202,20010309,-0.494888,-0.480540,123.47,-0.244510,-0.468421,-1.430906,-1.471366,0.240059,0.297527,-1.636192,-0.013111,-0.333621,-0.270195,-0.110353,-1.609645,-0.027182,-1
3,203,20010312,-0.609195,-0.550982,117.78,-0.285677,-0.534586,-3.406283,-3.679961,1.023836,1.177954,-1.982363,-0.009977,-0.340270,-0.274561,-0.112813,-1.609645,-0.046084,-1
4,204,20010313,-0.615233,-0.596905,119.72,-1.009871,-0.601999,0.670392,0.718906,0.307240,0.396222,-2.125731,-0.012221,-0.346038,-0.277851,-0.114912,-1.036723,0.016471,1


In [5]:
#check whether label1 has 2 classes
print( df['label1'][1:].unique())

[ 1 -1  0]


In [6]:
#prepare data for DT
X = df[['low', 'high', 'close','open','volume','close-open_%of_close','high-low_%of_close',
             'macd', 'rsi', '50d_avg', '100d_avg', '200d_avg', 'so']][:-1]
Y = df['label1'][1:]
Y = Y.apply(np.int64)
len(X)==len(Y)

True

In [7]:
#split the training and testing data
X_train = X.head(3000)
y_train = Y.head(3000)

X_test = X.loc[3000:]
y_test = Y.loc[3001:]
len(X_test) ==len(y_test)

True

In [8]:
#train
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [9]:
#predict
y_test_hat = clf.predict(X_test)
y_train_hat = clf.predict(X_train)

In [10]:
#evaluate
#build dataframes to compare actual class and predicted class
df_test_comp = pd.DataFrame(data = np.asarray(y_test), columns=['y_test'])
df_test_comp['y_test_hat'] = y_test_hat

df_train_comp = pd.DataFrame(data = np.asarray(y_train), columns=['y_train'])
df_train_comp['y_train_hat'] = y_train_hat


In [11]:
#calculate accuracy rate, both training and testing
df_test_comp['accuracy_level'] = np.where(df_test_comp['y_test'] == df_test_comp['y_test_hat'], 'correct', 'NA')
numcorrect = df_test_comp['accuracy_level'].loc[df_test_comp['accuracy_level']=='correct'].count()

df_test_comp['accuracy_level'] = np.where(df_test_comp['y_test'] > df_test_comp['y_test_hat'], 'bear', 'NA')
numbear = df_test_comp['accuracy_level'].loc[df_test_comp['accuracy_level']=='bear'].count()

df_test_comp['accuracy_level'] = np.where(df_test_comp['y_test'] < df_test_comp['y_test_hat'], 'bull', 'NA')
numbull = df_test_comp['accuracy_level'].loc[df_test_comp['accuracy_level']=='bull'].count()

In [12]:
df_test_comp['direction_correct'] = 0

In [13]:
#check if directional correct
for i in range(0, len(df_test_comp)):
    if df_test_comp['y_test'][i] * df_test_comp['y_test_hat'][i] > 0:
        df_test_comp['direction_correct'][i] = 1
    else:
        pass


/home/shiyun/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:
#total number of correct prediction in the test set
numcorrect

In [ ]:
#total number of BULL prediction in the test set
numbull

In [ ]:
#accura
df_test_comp['direction_correct'].sum()/len(df_test_comp)

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file='tree.dot') 
graph = graphviz.Source(dot_data) 


In [21]:
#confusion matrix
df_test_comp['bull-bull'] = np.where(df_test_comp['y_test']+df_test_comp['y_test_hat']==2, 1,0)
df_test_comp['bull-bear'] = np.where(df_test_comp['y_test'] > df_test_comp['y_test_hat'], 1,0)
df_test_comp['bear-bear'] = np.where(df_test_comp['y_test'] + df_test_comp['y_test_hat']==-2, 1,0)
df_test_comp['bear-bull'] = np.where(df_test_comp['y_test'] < df_test_comp['y_test_hat'], 1,0)

In [22]:
df_test_comp.head()

,y_test,y_test_hat,accuracy_level,direction_correct,bull-bull,bull-bear,bear-bear,bear-bull
0,1,-1,NA,0,0,1,0,0
1,1,-1,NA,0,0,1,0,0
2,1,-1,NA,0,0,1,0,0
3,-1,-1,NA,1,0,0,1,0
4,1,1,NA,1,1,0,0,0


In [29]:
bull_bear = df_test_comp['bull-bear'].sum()/len(df_test_comp)
bull_bull= df_test_comp['bull-bull'].sum()/len(df_test_comp)
bear_bear= df_test_comp['bear-bear'].sum()/len(df_test_comp)
bear_bull= df_test_comp['bear-bull'].sum()/len(df_test_comp)

In [30]:
bull_bear

0.4365781710914454

In [31]:
bull_bull

0.11135693215339233

In [32]:
bear_bear

0.3502949852507375

In [33]:
bear_bull

0.10176991150442478